In [3]:
import open3d as o3d
import numpy as np

# .ply fájl megnyitása
def read_ply_file(file_path):
    # Open3D használata a .ply fájl olvasására
    point_cloud = o3d.io.read_point_cloud(file_path)
    return point_cloud

# Példa használat
file_path = 'centered.ply'  # A .ply fájl elérési útja
point_cloud = read_ply_file(file_path)  # A pontfelhő kinyerése a fájlból

# Csúcsok kinyerése a pontfelhőből
vertices = np.asarray(point_cloud.points)

# Távolságok kiszámítása
distances = np.linalg.norm(vertices, axis=1)

# Távolságok normalizálása
normalized_distances = (distances - np.min(distances)) / (np.max(distances) - np.min(distances))

# Színek hozzárendelése
colors = np.repeat(normalized_distances[:, np.newaxis], 3, axis=1)
colors = 1 - colors  # Invertáljuk, hogy a közelebbi pontok sötétebbek legyenek

# Színek hozzáadása a pontfelhőhöz
point_cloud.colors = o3d.utility.Vector3dVector(colors)

# Pontfelhő leminősítése (voxel down-sampling)
point_cloud = point_cloud.voxel_down_sample(voxel_size=0.05)

# Normálvektorok számítása (szükséges a BPA-hez)
point_cloud.estimate_normals()
point_cloud.orient_normals_consistent_tangent_plane(k=20)

# Ball Pivoting Algorithm
distances = point_cloud.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist  # A golyó sugara

# Sugarak beállítása
radii = [radius, radius * 1.5]  # Példa: több sugár
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    point_cloud,
    o3d.utility.DoubleVector(radii))

# Háló simítása
mesh = mesh.filter_smooth_taubin(number_of_iterations=10)
mesh.compute_triangle_normals()  # Háromszög normálok újraszámítása
mesh.compute_vertex_normals()  # Csúcsnormálok újraszámítása

# Pontfelhő és háló együttes megjelenítése
o3d.visualization.draw_geometries(
    [mesh],
    mesh_show_back_face=True  # Hátsó oldalak megjelenítése
)


Ez itt fent 7 percces kb és egészen pontos surfacet csinál.

GPU-s:

In [1]:
import cupy as cp

if cp.is_available():
    print("✅ Fut GPU-n! (CuPy verzió:", cp.__version__, ")")
    print("GPU memória használat:", cp.cuda.Device().mem_info)
else:
    print("❌ Nem fut GPU-n :(")

✅ Fut GPU-n! (CuPy verzió: 13.4.1 )
GPU memória használat: (1727512167, 2147352576)


A Mesh nem jelenik meg.

In [2]:
import open3d as o3d
import numpy as np
from joblib import Parallel, delayed
import cupy as cp  # Requires CUDA-enabled GPU

def read_ply_file(file_path):
    """Read PLY file using Open3D with potential GPU acceleration"""
    point_cloud = o3d.io.read_point_cloud(file_path)
    return point_cloud

def gpu_distance_calculation(vertices):
    """Calculate distances using GPU acceleration"""
    print("⏳ Converting data to GPU...")
    vertices_gpu = cp.asarray(vertices)
    print("✅ Data on GPU. Calculating distances...")
    distances_gpu = cp.linalg.norm(vertices_gpu, axis=1)
    print("✅ Distances calculated. Copying back to CPU...")
    return cp.asnumpy(distances_gpu)

def parallel_normalize(distances):
    """Parallel distance normalization"""
    min_dist = np.min(distances)
    max_dist = np.max(distances)
    normalized = (distances - min_dist) / (max_dist - min_dist)
    return normalized

def process_chunk(vertices_chunk):
    """Process a chunk of vertices in parallel"""
    distances_chunk = np.linalg.norm(vertices_chunk, axis=1)
    return distances_chunk

def main():
    # Load point cloud
    file_path = 'centered.ply'
    point_cloud = read_ply_file(file_path)
    vertices = np.asarray(point_cloud.points)
    print(f"Loaded point cloud with {len(vertices)} points")

    # Option 1: GPU-accelerated distance calculation
    try:
        distances = gpu_distance_calculation(vertices)
    except Exception as e:
        print(f"GPU acceleration failed: {e}, falling back to CPU")
        num_cores = -1  # Use all available cores
        chunk_size = len(vertices) // (num_cores if num_cores > 0 else 1)
        results = Parallel(n_jobs=num_cores)(
            delayed(process_chunk)(vertices[i:i+chunk_size])
            for i in range(0, len(vertices), chunk_size)
        )
        distances = np.concatenate(results)

    # Normalize distances
    normalized_distances = parallel_normalize(distances)

    # Color mapping
    colors = np.repeat(normalized_distances[:, np.newaxis], 3, axis=1)
    colors = 1 - colors
    point_cloud.colors = o3d.utility.Vector3dVector(colors)
    print("Color mapping applied")

    # Downsampling
    point_cloud = point_cloud.voxel_down_sample(voxel_size=0.1)
    print(f"Points after downsampling: {len(point_cloud.points)}")

    # Normal estimation
    point_cloud.estimate_normals()
    point_cloud.orient_normals_consistent_tangent_plane(k=20)
    print("Normals estimated")

    # Ball Pivoting Algorithm
    distances = point_cloud.compute_nearest_neighbor_distance()
    avg_dist = np.mean(distances)
    radius = 3 * avg_dist
    radii = [radius, radius * 1.5]
    print(f"Average neighbor distance: {avg_dist:.4f}")
    print(f"Radii used for BPA: {radii}")  # MOVED THIS AFTER radii IS DEFINED
    
    # Mesh creation
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
        point_cloud,
        o3d.utility.DoubleVector(radii))
    print(f"Mesh created with {len(mesh.vertices)} vertices")

    # Mesh smoothing
    mesh = mesh.filter_smooth_taubin(number_of_iterations=10)
    mesh.compute_triangle_normals()
    mesh.compute_vertex_normals()
    print("Mesh smoothing completed")

    # Visualization
    print("Showing point cloud...")
    o3d.visualization.draw_geometries([point_cloud])
    
    print("Showing mesh...")
    o3d.visualization.draw_geometries(
        [mesh],
        mesh_show_back_face=True
    )

if __name__ == "__main__":
    main()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Loaded point cloud with 395700 points
⏳ Converting data to GPU...
✅ Data on GPU. Calculating distances...
✅ Distances calculated. Copying back to CPU...
Color mapping applied
Points after downsampling: 114093
Normals estimated
Average neighbor distance: 0.0745
Radii used for BPA: [0.2235956186910567, 0.33539342803658506]
Mesh created with 114093 vertices
Mesh smoothing completed
Showing point cloud...
Showing mesh...


In [7]:
point_cloud = read_ply_file("centered.ply")
print("Number of points:", len(point_cloud.points))

Number of points: 395700


jó ez, csak 20 perc

In [4]:
import open3d as o3d
import numpy as np
import cupy as cp
from numba import njit, prange

@njit(parallel=True)
def compute_colors_numba(vertices):
    min_dist = np.min(vertices)
    max_dist = np.max(vertices)
    colors = np.empty((vertices.shape[0], 3))
    for i in prange(vertices.shape[0]):
        val = 1 - (vertices[i] - min_dist) / (max_dist - min_dist)
        colors[i] = (val, val, val)
    return colors

def compute_colors_gpu(vertices):
    vertices_gpu = cp.asarray(vertices)
    min_dist = cp.min(vertices_gpu)
    max_dist = cp.max(vertices_gpu)
    normalized = 1 - (vertices_gpu - min_dist) / (max_dist - min_dist)
    colors = cp.stack((normalized, normalized, normalized), axis=1)
    return cp.asnumpy(colors)

input_file = "centered.ply"
pcd = o3d.io.read_point_cloud(input_file)
vertices = np.asarray(pcd.points)

distances = cp.linalg.norm(cp.asarray(vertices), axis=1).get()
colors = compute_colors_gpu(distances)
pcd.colors = o3d.utility.Vector3dVector(colors)

pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
pcd.orient_normals_consistent_tangent_plane(k=30)

distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radii = o3d.utility.DoubleVector([avg_dist*3, avg_dist*6])

mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, radii)
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)